参考: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

ここまで様々なフィードフォワードニューラルネットワークを見てきました。これまでのでは状態を保存する機能はありませんでした。  
シーケンスモデルは自然言語処理でよく使われます。これは入力の時系列を考慮できます。  
Hiddden Markov Modelとかconditional random fieldとかがよく使われてきました。  

RNNは状態を保存することができるネットワークです。例えば、ある時刻の出力を次の時刻の入力とすることで、情報が時を跨いで伝搬されます。  
LSTMでは、シーケンス中の各要素は、隠れ内部状態hというのがあります。これは任意の前の時刻の情報を得られます。  
language modelやpart-of-speech tagsやmyriadの場合に内部状態というのが使えます。

# LSTM

PyTorchのLSTMは入力として3次元のテンソルを受け取ります。各軸について把握しておきましょう。  
第1軸はシーケンス、第2軸はミニバッチ内のインデックス、そして第3軸は入力の要素です。  
まだミニバッチについて言及していないので、ここでは第2軸は1次元とします。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [13]:
lstm=nn.LSTM(3,3) # 入力の次元は3、出力の次元は3
inputs=[torch.randn(1,3) for _ in range(5)] # 長さ5のシーケンスを作る、各シーケンスは(1,3)の形を持つ

# 内部状態を初期化する
hidden=(torch.randn(1,1,3),torch.randn(1,1,3))

# 各入力について
for i in inputs:
    # シーケンスの各時刻について実行する
    # 各ステップの後、hiddenに内部状態が入る
    out,hidden=lstm(i.view(1,1,-1),hidden)

代わりに、一度に複数のシーケンスを入れることもできます。

In [14]:
# 一つ目の返り値は、シーケンス中の全ての内部状態
# 二つ目の返り値は、最後の内部状態
# outの最後の要素とhiddenは一緒
# outはシーケンス中の全ての内部状態を確認できる
# hiddenは後のLSTMで引数として与えることで、シーケンスを続けたり逆伝搬をしたりするためのもの
# ミニバッチに相当する第2次元を追加する
inputs=torch.cat(inputs).view(len(inputs),1,-1) # 入力
hidden=(torch.randn(1,1,3),torch.randn(1,1,3)) # 内部状態

print("hidden before running = ",hidden)

out,hidden=lstm(inputs,hidden) # LSTM

print("out = ",out)
print("hidden after running = ",hidden)

hidden before running =  (tensor([[[ 0.0085,  0.0678, -0.2043]]]), tensor([[[-0.5243, -0.3088,  0.4444]]]))
out =  tensor([[[-0.0123,  0.0384,  0.3112]],

        [[ 0.0533,  0.1213,  0.2932]],

        [[ 0.0745,  0.1729,  0.3718]],

        [[-0.0109,  0.2084,  0.4683]],

        [[-0.0401,  0.2125,  0.4205]]], grad_fn=<StackBackward>)
hidden after running =  (tensor([[[-0.0401,  0.2125,  0.4205]]], grad_fn=<StackBackward>), tensor([[[-0.1283,  0.4025,  0.6685]]], grad_fn=<StackBackward>))


# Part-of-Speech Tagging

speech tagを取得するためにLSTMを使います。ここではViterbiやForward-Backwardのようなものは使いません。  
まずはデータを用意します。

In [16]:
def prepare_sequence(seq,to_ix): # シーケンスを用意する関数
    idxs=[to_ix[w] for w in seq]
    
    return torch.tensor(idxs,dtype=torch.long)

In [17]:
# 文章とタグの組み合わせ
training_data=[
    ("The dog ate the apple".split(),["DET","NN","V","DET","NN"]),
    ("Everybody read that book".split(),["NN","V","DET","NN"])
]

word_to_ix={} # 語彙、ワード→番号のdict

for sent,tags in training_data: # 文章とタグ
    for word in sent: # 文章中の各単語について
        if word not in word_to_ix: # 語彙の中に含まれていないなら、新たに追加
            word_to_ix[word]=len(word_to_ix) # ワード→番号
        
print(word_to_ix)

tag_to_ix={"DET":0,"NN":1,"V":2} # 品詞→番号

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


モデルを定義します。

In [19]:
class LSTMTagger(nn.Module):
    def __init__(self,embedding_dim,hidden_dim,vocab_size,target_size):
        super(LSTMTagger,self).__init__()
        self.hidden_dim=hidden_dim # 隠れ層の次元
        self.word_embeddings=nn.Embedding(vocab_size,embedding_dim) # 埋め込み層の次元
        
        # LSTMはword embeddingを入力として受け取り、内部状態hiddenとその次元hidden_dimを出力する
        self.lstm=nn.LSTM(embedding_dim,hidden_dim)
        
        # 隠れ層から出力の次元までのマッピング
        self.hidden2tag=nn.Linear(hidden_dim,target_size)
        
    def forward(self,sentence):
        embeds=self.word_embeddings(sentence) # 埋め込み層
        lstm_out,_=self.lstm(embeds.view(len(sentence),1,-1)) # embeddingを入力として受け取る
        tag_space=self.hidden2tag(lstm_out.view(len(sentence),-1)) # タグの次元に変換
        tag_scores=F.log_softmax(tag_space,dim=1) # タグの予測値
        
        return tag_scores

それではモデルを学習させ、、、る前に、スコアを見ておきます。

In [20]:
# 大体32か64次元
# 小さくすることで、重みが訓練中にどう変化したかを見ることができる
EMBEDDING_DIM=6 # 埋め込み層の次元
HIDDEN_DIM=6 # 隠れ層の次元

model=LSTMTagger(EMBEDDING_DIM,HIDDEN_DIM,len(word_to_ix),len(tag_to_ix)) # モデル
loss_function=nn.NLLLoss() # ロス関数
optimizer=optim.SGD(model.parameters(),lr=0.1) # 最適化関数

# 学習の前のスコアを見ておく
with torch.no_grad(): # 勾配は計算しない
    inputs=prepare_sequence(training_data[0][0],word_to_ix) # これからモデルに入力するデータを用意する
    tag_scores=model(inputs) # モデルの出力
    print(tag_scores)

tensor([[-1.1460, -1.1694, -0.9901],
        [-1.1090, -1.1806, -1.0133],
        [-1.2453, -1.0405, -1.0248],
        [-1.1733, -1.1479, -0.9852],
        [-1.1454, -1.2117, -0.9565]])


次に学習させてみます。その後に改めてスコアを見てみます。

In [28]:
for epoch in range(300): # エポックを繰り返す
    for sentence,tags in training_data: # 訓練データ中
        # PyTorchでは勾配が蓄積されてゆくので、最初に初期化する
        model.zero_grad() 
        
        # ネットワークに入れるための入力を取得する
        # ワードの番号のテンソル
        sentence_in=prepare_sequence(sentence,word_to_ix) # 入力する文章
        targets=prepare_sequence(tags,tag_to_ix) # ターゲット
        
        # 順伝搬させる
        tag_scores=model(sentence_in) # モデルの出力
        
        # ロス、勾配を計算し、パラメータを更新する
        loss=loss_function(tag_scores,targets) # ロス
        loss.backward() # 逆伝搬させる
        optimizer.step() # パラメータを更新する

# 学習の後のスコアを見る
with torch.no_grad():
    inputs=prepare_sequence(training_data[0][0],word_to_ix) # 入力
    tag_scores=model(inputs) # モデルの出力
    
    # 文章は"the dog ate the apple"
    # i,jはそれぞれワードiのタグj
    # 予測タグは予測スコアの最大のもの
    print(training_data[0][0])
    print(tag_scores)
    print(tag_scores.argmax(axis=1))

['The', 'dog', 'ate', 'the', 'apple']
tensor([[-9.8161e-03, -5.4920e+00, -5.1764e+00],
        [-6.7108e+00, -3.8032e-03, -5.9607e+00],
        [-4.4137e+00, -6.2204e+00, -1.4198e-02],
        [-1.0209e-02, -5.6630e+00, -5.0078e+00],
        [-4.7409e+00, -9.2672e-03, -7.6145e+00]])
tensor([0, 1, 2, 0, 1])


例えば↑では、予測クラスは[0,1,2,0,1]となります。  
ここで{"DET":0,"NN":1,"V":2}です。